In [1]:
import polars as pl

In [2]:
df = pl.read_csv("https://data.cityofnewyork.us/resource/h9gi-nx95.csv")

In [3]:
df.head(2)

crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,cross_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
str,str,str,i64,f64,f64,str,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,str,str,str,str,str,i64,str,str,str,str,str
"""2021-09-11T00:…","""2:39""",null,null,null,null,null,"""WHITESTONE EXP…","""20 AVENUE""",null,2,0,0,0,0,0,2,0,"""Aggressive Dri…","""Unspecified""",null,null,null,4455765,"""Sedan""","""Sedan""",null,null,null
"""2022-03-26T00:…","""11:45""",null,null,null,null,null,"""QUEENSBORO BRI…",null,null,1,0,0,0,0,0,1,0,"""Pavement Slipp…",null,null,null,null,4513547,"""Sedan""",null,null,null,null


In [34]:
df.schema

{'crash_date': Utf8,
 'crash_time': Utf8,
 'borough': Utf8,
 'zip_code': Int64,
 'latitude': Float64,
 'longitude': Float64,
 'location': Utf8,
 'on_street_name': Utf8,
 'off_street_name': Utf8,
 'cross_street_name': Utf8,
 'number_of_persons_injured': Int64,
 'number_of_persons_killed': Int64,
 'number_of_pedestrians_injured': Int64,
 'number_of_pedestrians_killed': Int64,
 'number_of_cyclist_injured': Int64,
 'number_of_cyclist_killed': Int64,
 'number_of_motorist_injured': Int64,
 'number_of_motorist_killed': Int64,
 'contributing_factor_vehicle_1': Utf8,
 'contributing_factor_vehicle_2': Utf8,
 'contributing_factor_vehicle_3': Utf8,
 'contributing_factor_vehicle_4': Utf8,
 'contributing_factor_vehicle_5': Utf8,
 'collision_id': Int64,
 'vehicle_type_code1': Utf8,
 'vehicle_type_code2': Utf8,
 'vehicle_type_code_3': Utf8,
 'vehicle_type_code_4': Utf8,
 'vehicle_type_code_5': Utf8}

In [26]:
(df.groupby("collision_id")
   .count()
   .filter(pl.col("count") > 1))

collision_id,count
i64,u32


In [7]:
# df.with_columns(
#     pl.col("crash_date").

In [7]:
df.groupby("borough").count()

borough,count
str,u32
"""QUEENS""",154
"""BRONX""",107
"""MANHATTAN""",98
"""STATEN ISLAND""",27
"""BROOKLYN""",247
null,367


In [9]:
nn_df = df.filter(pl.col("borough").is_not_null())

In [20]:
df.filter(pl.col("borough").is_not_null()).select("borough").unique()

borough
str
"""QUEENS"""
"""MANHATTAN"""
"""STATEN ISLAND"""
"""BRONX"""
"""BROOKLYN"""


In [4]:
borough_df = pl.DataFrame({
                "borough": ["BROOKLYN", "BRONX", "MANHATTAN", "STATEN ISLAND", "QUEENS"],
                "population": [2590516, 1379946, 1596273, 2278029, 378977],
                "area":[179.7, 109.2, 58.68, 281.6, 149.0]
})

In [42]:
(df.filter(pl.col("borough").is_not_null())
   .select(["borough", "number_of_persons_injured"])
   .groupby("borough")
   .sum()
   .join(borough_df, on=["borough"])
   .select([
       "borough", 
       (pl.col("number_of_persons_injured") / pl.col("population")).alias("injuries_per_population")
   ])
)

borough,injuries_per_population
str,f64
"""BROOKLYN""",0.000045
"""BRONX""",0.000033
"""MANHATTAN""",0.000025
"""STATEN ISLAND""",0.000007
"""QUEENS""",0.000193


In [5]:
new_df = df.join(borough_df, on=["borough"], how="left")

In [9]:
new_df.write_parquet("s3://harmonskis/nyc_accidents.parquet")

thread '<unnamed>' panicked at 'called `Result::unwrap()` on an `Err` value: Os { code: 2, kind: NotFound, message: "No such file or directory" }', src/dataframe.rs:713:46
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace


PanicException: called `Result::unwrap()` on an `Err` value: Os { code: 2, kind: NotFound, message: "No such file or directory" }